In [1]:
import osrm
import requests
from fake_headers import Headers
from bs4 import BeautifulSoup
from selenium import webdriver as wd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pathlib import Path
from tqdm import tqdm
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import random
import time
from pathlib import Path
import shutil
from copy import deepcopy

from src.utils.main_parser import GeoProcessor

In [2]:
html = requests.get('https://alta.ru/railway')

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
russia = soup.find('div', class_='pRailway_block')
rzd_lines = [f'https://alta.ru{i["href"]}' for i in russia.find_all('a', class_='pRailway_item')]
rzd_lines_names = [i.text for i in russia.find_all('a', class_='pRailway_item')]

In [5]:
stations_data = []
for rzd_line, rzd_line_name in tqdm(zip(rzd_lines, rzd_lines_names), total=len(rzd_lines)):
    for _ in range(5):
        try:
            rzd_line = requests.get(rzd_line, timeout=10)
            line_soup = BeautifulSoup(rzd_line.text, 'html.parser')
            all_stations_block = line_soup.find('div', class_='pRailway_blocks js-masonry')
            all_stations = all_stations_block.find_all('a', class_='pRailway_item mFastSearch_key')
            
            names = [i.text for i in all_stations]
            hrefs = [f'https://alta.ru{i["href"]}' for i in all_stations]
            for st_name, href in zip(names, hrefs):
                stations_data.append([st_name, href, rzd_line_name])
            time.sleep(1)
            break
        except:
            pass

100%|██████████| 20/20 [00:24<00:00,  1.22s/it]


In [6]:
rzd_stations = pd.DataFrame(stations_data)
rzd_stations.columns = ['station_name', 'href', 'rzd_line_name']

In [7]:
rzd_stations

,station_name,href,rzd_line_name
0,Азей,https://alta.ru/railway/station/92220/,Восточно-Сибирская железная дорога
1,Акульшет,https://alta.ru/railway/station/92004/,Восточно-Сибирская железная дорога
2,Алгашет,https://alta.ru/railway/station/92082/,Восточно-Сибирская железная дорога
3,Алзамай,https://alta.ru/railway/station/92070/,Восточно-Сибирская железная дорога
4,Ангаракан (рзд),https://alta.ru/railway/station/90392/,Восточно-Сибирская железная дорога
...,...,...,...
5237,Рассолода,https://alta.ru/railway/station/91440/,Якутская железная дорога
5238,Тенистый (рзд),https://alta.ru/railway/station/91380/,Якутская железная дорога
5239,Томмот,https://alta.ru/railway/station/91420/,Якутская железная дорога
5240,Чульбас (рзд),https://alta.ru/railway/station/91370/,Якутская железная дорога


parse one rzd station

In [9]:
rzd_stations['code'] = 'nan'
rzd_stations['coords'] = 'nan'

# start point

In [7]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(filename='station_parsing.log', level=logging.INFO)
def parse_station_href(station_href: str):
    rzd_code = 'pRailway_column pRailway_column-left pRailway_column-esr' # <br>
    rzd_lat = 'dib pr-10' # <br>
    rzd_lon = 'dib pl-10' # <br>

    one_station_html = requests.get(station_href)
    one_soup = BeautifulSoup(one_station_html.text, 'html.parser')
    st_code = one_soup.find('div', class_=rzd_code).text.split(':')[1].strip()
    st_lat = one_soup.find('div', class_=rzd_lat).text.split(':')[1].strip()
    st_lon = one_soup.find('div', class_=rzd_lon).text.split(':')[1].strip()
    return (st_code, ','.join([st_lat, st_lon]))

In [19]:
rzd_stations = pd.read_csv('rzd_stations.csv')
rzd_stations

,station_name,href,rzd_line_name,code,coords
0,Азей,https://alta.ru/railway/station/92220/,Восточно-Сибирская железная дорога,92220.0,"54.476325,100.837279"
1,Акульшет,https://alta.ru/railway/station/92004/,Восточно-Сибирская железная дорога,92004.0,"55.907636,98.097509"
2,Алгашет,https://alta.ru/railway/station/92082/,Восточно-Сибирская железная дорога,92082.0,"55.47171,98.620787"
3,Алзамай,https://alta.ru/railway/station/92070/,Восточно-Сибирская железная дорога,92070.0,"55.543184,98.661518"
4,Ангаракан (рзд),https://alta.ru/railway/station/90392/,Восточно-Сибирская железная дорога,90392.0,"56.325419,113.240286"
...,...,...,...,...,...
5237,Рассолода,https://alta.ru/railway/station/91440/,Якутская железная дорога,NaN,NaN
5238,Тенистый (рзд),https://alta.ru/railway/station/91380/,Якутская железная дорога,NaN,NaN
5239,Томмот,https://alta.ru/railway/station/91420/,Якутская железная дорога,NaN,NaN
5240,Чульбас (рзд),https://alta.ru/railway/station/91370/,Якутская железная дорога,NaN,NaN


In [20]:
rzd_stations[rzd_stations.code.isna()]

,station_name,href,rzd_line_name,code,coords
35,Диабазовый,https://alta.ru/railway/station/92650/,Восточно-Сибирская железная дорога,NaN,NaN
164,Уланмакит,https://alta.ru/railway/station/90600/,Восточно-Сибирская железная дорога,NaN,NaN
223,Боголюбово,https://alta.ru/railway/station/26272/,Горьковская железная дорога,NaN,NaN
225,Болдино,https://alta.ru/railway/station/26221/,Горьковская железная дорога,NaN,NaN
253,Ворсма,https://alta.ru/railway/station/26920/,Горьковская железная дорога,NaN,NaN
...,...,...,...,...,...
5237,Рассолода,https://alta.ru/railway/station/91440/,Якутская железная дорога,NaN,NaN
5238,Тенистый (рзд),https://alta.ru/railway/station/91380/,Якутская железная дорога,NaN,NaN
5239,Томмот,https://alta.ru/railway/station/91420/,Якутская железная дорога,NaN,NaN
5240,Чульбас (рзд),https://alta.ru/railway/station/91370/,Якутская железная дорога,NaN,NaN


In [21]:
global_break_flag = False
rzd_save_name = 'rzd_stations.csv'
part_table = rzd_stations[rzd_stations.code.isna()]
for idx, station_href in tqdm(zip(part_table.index.tolist(), part_table.href.values), total=part_table.shape[0]):
    break_flag = False
    for i in range(2):
        if not break_flag:
            try:
                parsed = parse_station_href(station_href)
                try:
                    rzd_stations.loc[idx, 'code'] = parsed[0]
                except:
                    logging.info(f'{idx=}, code is wrong')
                    rzd_stations.loc[idx, 'code'] = 'nan'
                try:
                    rzd_stations.loc[idx, 'coords'] = parsed[1]
                except:
                    logging.info(f'{idx=}, coords is wrong')
                    rzd_stations.loc[idx, 'coords'] = 'nan'
                break_flag = True
                # logging.info(f'{idx=} {station_href=} is succesfully parsed')
            except KeyboardInterrupt:
                global_break_flag = True
                break
            except Exception as e:
                logging.info(f'{idx=}, smth went wrong: {str(e)}')
            time.sleep(1.)
        else:
            break
        time.sleep(1.5)
    if global_break_flag:
        break
    if idx % 10 == 0:
        logging.info(f'{idx=} csv was saved to {rzd_save_name}')
        rzd_stations.to_csv(rzd_save_name, index=False, encoding='UTF-8')

  0%|          | 4/4498 [00:27<8:35:11,  6.88s/it]


KeyboardInterrupt: 